# Here i want to extract from lichess data features important for future modeling

So here i want to get data in format similar to that: 

columns = ( date, amount of mathes played in that date,ammont of matches played day earllier , amount of matches plyed for previous mounth, ranikng points fluctuation (day vs day before), ranking after last match that day, mean oponent vs my ranking diff that day ) and maybe some other columns that i will consider as important for ranking changes prediction. --> df2

And all of those for different times control (blitz, rapid, bullet)

and other data for winner prediction models, similar columns as without any chnages but some new cols like: time used in game, time for move and maybe some other --> df1

# To Be Continued

In [75]:
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import plot_tree

from sklearn.model_selection import train_test_split
# Regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
# Clasification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, accuracy_score
# Best_searches
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# Standarization
from sklearn.preprocessing import StandardScaler

from keras import models
from keras.layers import Dense

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import pandas as pd
import numpy as np
import arrow
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv("Chess_data_basic.csv", index_col = "Unnamed: 0")

In [5]:
len(set(df["black_player_name"]+df["white_player_name"]))

4789

In [6]:
df.isna().sum()

id                     0
rated                  0
variant                0
speed                  0
createdAt              0
lastMoveAt             0
status                 0
winner               250
moves                  1
clock                  0
movesperplayer         0
white_player_name      0
white_ranking          0
black_player_name      0
black_ranking          0
dtype: int64

In [7]:
def convert_date_with_offset(date_str):
    return arrow.get(date_str).datetime

for i in ['createdAt', 'lastMoveAt']:
    df[i] = df[i].apply(convert_date_with_offset)
    
# Create Gametime column
df["Gametime"] = df["lastMoveAt"] - df["createdAt"]
# Handle Gametime column
df['Gametime'] = pd.to_timedelta(df['Gametime'])

rounding = lambda x: round(x, 1)

df['Gametime'] = df['Gametime'].dt.total_seconds().apply(rounding)
df['timepermove']=(df['Gametime']/df["movesperplayer"]).apply(rounding)

next think to do is taking insight from moves column, there is a lot of possibility but i will concentrate on:
1) cheking if side castle and if long or short castle
2) taking number of moves before castle ( if no castle - values set to number of moves +1 )
3) groping by opening (for this project i will take in count only first 6 moves - 3 moves per side )
4) number of checks in first 10 moves ( per side )

## Handle NA values

In [8]:
# Only timepermove colum had NA values (just one), expect winner, where NA == DRAW
df.dropna(subset=['timepermove'],inplace=True)

df = df.fillna("draw")

df.isna().sum()

id                   0
rated                0
variant              0
speed                0
createdAt            0
lastMoveAt           0
status               0
winner               0
moves                0
clock                0
movesperplayer       0
white_player_name    0
white_ranking        0
black_player_name    0
black_ranking        0
Gametime             0
timepermove          0
dtype: int64

# Checking for side castle 

kingside castle ( short ) is O-O in moves notation and queenside castle ( long ) is O-O-O

In [9]:
pd.options.mode.chained_assignment = None

In [10]:
def moves_column(df):
    df["moves"] = df["moves"].str.split(" ")

    df["white_moves"]=df["moves"].apply(lambda x: x[::2])
    df["black_moves"]=df["moves"].apply(lambda x: x[1::2])
    return df

df = moves_column(df)

In [11]:
def check_for_queen_or_king_castle(df):
    check_for_kingcastle = lambda x: 1 if "O-O" in x else 0
    check_for_queencastle = lambda x: 1 if "O-O-O" in x else 0
    df["white_kingcastle"] = df["white_moves"].apply(check_for_kingcastle)
    df["white_queencastle"] = df["white_moves"].apply(check_for_queencastle)
    df["black_kingcastle"] = df["black_moves"].apply(check_for_kingcastle)
    df["black_queencastle"] = df["black_moves"].apply(check_for_queencastle)
    return df

df = check_for_queen_or_king_castle(df)

# num of moves before castle 

In [12]:
def find_index_of_first_occurrence(lst):
    elements = ["O-O", "O-O-O"]
    for element in elements:
        if element in lst:
            return lst.index(element)
    return len(lst) + 1

df["White_moves_before_castle"] = df["white_moves"].apply(find_index_of_first_occurrence)
df["Black_moves_before_castle"] = df["black_moves"].apply(find_index_of_first_occurrence)

In [13]:
df.to_csv("to_visual.csv")

In [14]:
df.to_json("to_visual1.json")

## Grouping by opening

In [15]:
df["open4"] = df["moves"].apply(lambda x: x[:4])
df["open4"] = df["open4"].apply(lambda x: " ".join(x))

In [16]:
white_opens = df[df['white_player_name'] == 'EvilSaintPL']["open4"].value_counts().head(20)
black_opens = df[df['black_player_name'] == 'EvilSaintPL']["open4"].value_counts().head(20)

In [17]:
def iterable_opening_list_and_my_ranking( df ):
    
    openings_list = []
    rank_of_evilsaintpl_list = []
    evilsaintpl_won = []
    enemy_rank_list = []

    
    for index, row in df.iterrows():
        
        # OPENS1
        if df["white_player_name"][index]=="EvilSaintPL" and df["open4"][index] in white_opens.index:
            openings_list.append("White " + df["open4"][index])
        elif  df["white_player_name"][index]!="EvilSaintPL" and df["open4"][index] in black_opens.index:
            openings_list.append("Black " + df["open4"][index]) 
        else:
            openings_list.append("no opens")
        
        # MY RANK AND ENEMY RANK 
        if df["white_player_name"][index]=="EvilSaintPL":
            rank_of_evilsaintpl_list.append(df["white_ranking"][index])
            enemy_rank_list.append(df["black_ranking"][index])
        elif df["black_player_name"][index]=="EvilSaintPL":
            rank_of_evilsaintpl_list.append(df["black_ranking"][index])
            enemy_rank_list.append(df["white_ranking"][index])
        else:
            print(index)
            
        # I WON 
        if df["winner"][index] == 'draw':
            evilsaintpl_won.append(0)
        elif df["white_player_name"][index]=="EvilSaintPL" and df["winner"][index]=='white':
            evilsaintpl_won.append(1)
        elif  df["black_player_name"][index]=="EvilSaintPL" and df["winner"][index]=='black':
            evilsaintpl_won.append(1)
        else:
            evilsaintpl_won.append(-1)

    df["opens1"]= openings_list
    df["my_rank"]= rank_of_evilsaintpl_list
    df['i_won'] = evilsaintpl_won
    df['enemy_rank'] = enemy_rank_list
    
    # NEW RANKS FE
    df['rank_difference'] = df['my_rank'] - df['enemy_rank']
    df['is_higher'] = (df['my_rank'] > df['enemy_rank'])


    return df

df = iterable_opening_list_and_my_ranking(df)

In [18]:
df.head(1)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open4,opens1,my_rank,i_won,enemy_rank,rank_difference,is_higher
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nc6,1753,1,1735,18,True


In [19]:
def clear_dataframe(df):
    temp = pd.get_dummies(df["opens1"])
    df = pd.concat([df,temp], axis=1)
    
    df.drop("status",axis=1, inplace=True)
    df.drop(labels = ["opens1"], axis=1, inplace=True)
#    df.drop("id",axis=1,inplace=True)
    
    df = df[df["rated"]==True]
    df = df[df["white_player_name"]!="YoungCapitan"]
    df = df[df["black_player_name"]!="YoungCapitan"]
    df.drop("variant",axis=1,inplace=True)
    
    return df

df = clear_dataframe(df)

In [20]:
df.head(1)

,id,rated,speed,createdAt,lastMoveAt,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open4,my_rank,i_won,enemy_rank,rank_difference,is_higher,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens
0,O83cQxfo,True,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,1,1735,18,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [21]:
df["i_won"]=df["i_won"]+1

In [22]:
df.shape

(4880, 70)

In [23]:
df.speed.unique()

array(['blitz', 'classical', 'bullet', 'rapid'], dtype=object)

In [24]:
def rankdifftest(df):
    
    blitz = df[df["speed"]=="blitz"].reset_index(drop=True)
    classical = df[df["speed"]=="classical"].reset_index(drop=True)
    bullet = df[df["speed"]=="bullet"].reset_index(drop=True)
    rapid = df[df["speed"]=="rapid"].reset_index(drop=True)
    
    for speed in [blitz, classical, bullet, rapid]:
        rank_resu=list()
        for index, row in speed.iterrows():
            if index==0:
                rank_resu.append(speed["my_rank"][index])
            else:
                rank_resu.append(speed["my_rank"][index-1])
        speed["rank_after_game"]=rank_resu
        speed["my_rank_diff"]=speed["rank_after_game"]-speed["my_rank"]
    return pd.concat([blitz, classical, bullet, rapid])

In [25]:
df = rankdifftest(df)

In [26]:
df.shape

(4880, 72)

In [27]:
df.head(2)

,id,rated,speed,createdAt,lastMoveAt,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open4,my_rank,i_won,enemy_rank,rank_difference,is_higher,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens,rank_after_game,my_rank_diff
0,O83cQxfo,True,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,2,1735,18,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,1753,0
1,KeBcqcHz,True,blitz,2023-06-28 02:31:45.081000+02:00,2023-06-28 02:33:30.178000+02:00,white,"[Nf3, Nc6, g3, e5, Bg2, e4, Ng1, d5, d3, f5, d...",300,18,convict-19,1801,EvilSaintPL,1758,105.1,5.8,"[Nf3, g3, Bg2, Ng1, d3, dxe4, Nc3, Bg5, Bxf6, ...","[Nc6, e5, e4, d5, f5, fxe4, Nf6, Be7, Bxf6, Qx...",1,0,0,1,17,14,Nf3 Nc6 g3 e5,1758,0,1801,-43,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,1753,-5


In [28]:
df.columns

Index(['id', 'rated', 'speed', 'createdAt', 'lastMoveAt', 'winner', 'moves',
       'clock', 'movesperplayer', 'white_player_name', 'white_ranking',
       'black_player_name', 'black_ranking', 'Gametime', 'timepermove',
       'white_moves', 'black_moves', 'white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle',
       'Black_moves_before_castle', 'open4', 'my_rank', 'i_won', 'enemy_rank',
       'rank_difference', 'is_higher', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4

In [29]:
final_all = df[[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher' ,'clock', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

blitz = df[df["speed"]=="blitz"][[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

classical = df[df["speed"]=="classical"][[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

bullet = df[df["speed"]=="bullet"][[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

rapid = df[df["speed"]=="rapid"][[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

In [30]:
print(f"blitz : {blitz.shape},\nclassic: {classical.shape},\nbullet: {bullet.shape},\nrapid: {rapid.shape},\nfull: {final_all.shape}")

blitz : (3417, 55),
classic: (153, 55),
bullet: (975, 55),
rapid: (335, 55),
full: (4880, 56)


In [31]:
blitz.head(2)

,i_won,my_rank,enemy_rank,rank_difference,is_higher,movesperplayer,Gametime,timepermove,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,no opens,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7
0,2,1753,1735,18,True,70,617.0,8.8,1,0,0,0,6,71,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,0,1758,1801,-43,False,18,105.1,5.8,1,0,0,1,17,14,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [32]:
rapid.head(2)

,i_won,my_rank,enemy_rank,rank_difference,is_higher,movesperplayer,Gametime,timepermove,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,no opens,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7
0,0,1790,1785,5,True,62,1079.0,17.4,1,0,1,0,10,9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,2,1777,1783,-6,False,37,845.4,22.8,0,1,1,0,10,8,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [33]:
df.speed.unique()

array(['blitz', 'classical', 'bullet', 'rapid'], dtype=object)

In [34]:
game_modes = [blitz, classical, bullet, rapid, final_all]
game_modes_fixed = []

def fix_infinity():
    for i in game_modes:
        i = i[~i["timepermove"].isin([np.inf, -np.inf])]
        game_modes_fixed.append(i)
    return game_modes_fixed

blitz, classical, bullet, rapid, final_all = fix_infinity()

In [35]:
# blitz = blitz[~blitz["timepermove"].isin([np.inf, -np.inf])]

In [36]:
bullet[bullet['timepermove'] > 1000]

,i_won,my_rank,enemy_rank,rank_difference,is_higher,movesperplayer,Gametime,timepermove,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,no opens,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7


In [37]:
# column definiton
"""
i_won - 0 -> I lost, 1-Draw, 2-I_Won
is_higher -> is my rank higher than my enemy
timepermove -> Gametime / movesperplayer
White_moves_before_castle -> amount of white moves before castle
no opens - no specyfic opening typed played by evilsainpl
"""

'\ni_won - 0 -> I lost, 1-Draw, 2-I_Won\nis_higher -> is my rank higher than my enemy\ntimepermove -> Gametime / movesperplayer\nWhite_moves_before_castle -> amount of white moves before castle\nno opens - no specyfic opening typed played by evilsainpl\n'

In [38]:
X = final_all[['my_rank', 'enemy_rank', 'rank_difference', 'is_higher', 'clock',
       'movesperplayer', 'Gametime', 'timepermove', 'white_kingcastle',
       'white_queencastle', 'black_kingcastle', 'black_queencastle',
       'White_moves_before_castle', 'Black_moves_before_castle', 'no opens',
       'Black c4 e5 Nc3 Nf6', 'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6',
       'Black d4 Nf6 c4 e6', 'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6',
       'Black e4 e5 Bc4 Nc6', 'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6',
       'Black e4 e5 Nf3 Nc6', 'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4',
       'Black e4 e5 f4 d5', 'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5',
       'Black e4 e6 Nc3 d5', 'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5',
       'Black e4 e6 e5 c5', 'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6',
       'White e4 c5 Nf3 d6', 'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5',
       'White e4 c6 d4 d5', 'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6',
       'White e4 d5 exd5 Qxd5', 'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6',
       'White e4 e5 Nc3 d6', 'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6',
       'White e4 e5 Nf3 Nf6', 'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5',
       'White e4 e5 Nf3 d6', 'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5',
       'White e4 g6 d4 Bg7']]

y = final_all['i_won']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [40]:
X.dtypes

my_rank                        int64
enemy_rank                     int64
rank_difference                int64
is_higher                       bool
clock                          int64
movesperplayer                 int64
Gametime                     float64
timepermove                  float64
white_kingcastle               int64
white_queencastle              int64
black_kingcastle               int64
black_queencastle              int64
White_moves_before_castle      int64
Black_moves_before_castle      int64
no opens                        bool
Black c4 e5 Nc3 Nf6             bool
Black d4 Nf6 Bf4 e6             bool
Black d4 Nf6 Nf3 e6             bool
Black d4 Nf6 c4 e6              bool
Black d4 d5 Bf4 Nf6             bool
Black d4 d5 c4 c6               bool
Black e4 e5 Bc4 Nc6             bool
Black e4 e5 Bc4 Nf6             bool
Black e4 e5 Nc3 Nf6             bool
Black e4 e5 Nf3 Nc6             bool
Black e4 e5 Nf3 Nf6             bool
Black e4 e5 d4 exd4             bool
B

In [41]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, precision_score, accuracy_score



In [72]:
def model_for_different_gamemodes():
    final_results = {}
    all_dataframes = [blitz, classical, bullet, rapid, final_all]
    dataframes_names_str = ["blitz", "classical", "bullet", "rapid", "all_data"]
    for dataframe, dataframe_name in zip(all_dataframes, dataframes_names_str):
        X = dataframe[['my_rank', 'enemy_rank', 'rank_difference', 'is_higher',
                       'movesperplayer', 'Gametime', 'timepermove', 'white_kingcastle',
                       'white_queencastle', 'black_kingcastle', 'black_queencastle',
                       'White_moves_before_castle', 'Black_moves_before_castle', 'no opens',
                       'Black c4 e5 Nc3 Nf6', 'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6',
                       'Black d4 Nf6 c4 e6', 'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6',
                       'Black e4 e5 Bc4 Nc6', 'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6',
                       'Black e4 e5 Nf3 Nc6', 'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4',
                       'Black e4 e5 f4 d5', 'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5',
                       'Black e4 e6 Nc3 d5', 'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5',
                       'Black e4 e6 e5 c5', 'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6',
                       'White e4 c5 Nf3 d6', 'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5',
                       'White e4 c6 d4 d5', 'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6',
                       'White e4 d5 exd5 Qxd5', 'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6',
                       'White e4 e5 Nc3 d6', 'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6',
                       'White e4 e5 Nf3 Nf6', 'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5',
                       'White e4 e5 Nf3 d6', 'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5',
                       'White e4 g6 d4 Bg7']]
        y = dataframe['i_won']
        # Define the parameter grid for grid search
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)        
        
        param_dist = {
            'n_estimators': np.arange(100, 500, 100),
            'max_depth': np.arange(1, 70, 7),
            'min_samples_leaf': np.arange(1, 70, 7),
            'min_samples_split': np.arange(1, 70, 7),
            'max_features': ['sqrt', 'log2']
        }

        # Create the Random Forest classifier
        rfc = RandomForestClassifier()

        # Create the randomized search object
        random_search = RandomizedSearchCV(rfc, param_distributions=param_dist, cv=5, n_iter=10)

        # Perform randomized search on the training data
        random_search.fit(X_train, y_train)

        # Get the best estimator and its corresponding parameters
        best_rf = random_search.best_estimator_
        print('Best Params: ', random_search.best_params_)

        # Fit the best estimator on the training data
        best_rf.fit(X_train, y_train)

        # Predict on the training and test data using the best estimator
        y_pred_train = best_rf.predict(X_train)
        y_pred = best_rf.predict(X_test)
                
        final_results[dataframe_name] = {}
        # Calculate the evaluation metrics
        final_results[dataframe_name]["Trainging Set metics"] = {}
        final_results[dataframe_name]["Trainging Set metics"]['f1_score'] = f1_score(y_train, y_pred_train, average='weighted')
        final_results[dataframe_name]["Trainging Set metics"]['precision_score'] = precision_score(y_train, y_pred_train, average='weighted')
        final_results[dataframe_name]["Trainging Set metics"]['accuracy_score'] = accuracy_score(y_train, y_pred_train)
        
        final_results[dataframe_name]["Test Set metics"] = {}
        final_results[dataframe_name]["Test Set metics"]['f1_score'] = f1_score(y_test, y_pred, average='weighted')
        final_results[dataframe_name]["Test Set metics"]['precision_score'] = precision_score(y_test, y_pred, average='weighted')
        final_results[dataframe_name]["Test Set metics"]['accuracy_score'] = accuracy_score(y_test, y_pred)      

    return final_results

In [73]:
final_res = model_for_different_gamemodes()

Best Params:  {'n_estimators': 200, 'min_samples_split': 50, 'min_samples_leaf': 43, 'max_features': 'sqrt', 'max_depth': 8}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best Params:  {'n_estimators': 200, 'min_samples_split': 57, 'min_samples_leaf': 64, 'max_features': 'log2', 'max_depth': 50}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best Params:  {'n_estimators': 400, 'min_samples_split': 15, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 50}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them

Best Params:  {'n_estimators': 100, 'min_samples_split': 36, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 8}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/sit

Best Params:  {'n_estimators': 100, 'min_samples_split': 57, 'min_samples_leaf': 29, 'max_features': 'sqrt', 'max_depth': 8}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [76]:
pprint.pprint(final_res)

{'all_data': {'Test Set metics': {'accuracy_score': 0.5512715340442986,
                                  'f1_score': 0.5250972261572464,
                                  'precision_score': 0.5351862890327607},
              'Trainging Set metics': {'accuracy_score': 0.602735978112175,
                                       'f1_score': 0.5787003589738385,
                                       'precision_score': 0.5854103058731519}},
 'blitz': {'Test Set metics': {'accuracy_score': 0.5433255269320844,
                               'f1_score': 0.523938863822786,
                               'precision_score': 0.5237711316722335},
           'Trainging Set metics': {'accuracy_score': 0.5939820242282141,
                                    'f1_score': 0.572082289484539,
                                    'precision_score': 0.5742988947873278}},
 'bullet': {'Test Set metics': {'accuracy_score': 0.47950819672131145,
                                'f1_score': 0.4696273702033889,
      

In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np

def model_for_different_gamemodes():
    final_results = {}
    all_dataframes = [blitz, classical, bullet, rapid, final_all]
    dataframes_names_str = ["blitz", "classical", "bullet", "rapid", "all_data"]
    classifiers = {
        'Random Forest': {
            'model': RandomForestClassifier(),
            'param_dist': {
                'n_estimators': np.arange(100, 500, 100),
                'max_depth': np.arange(1, 70, 7),
                'min_samples_leaf': np.arange(1, 70, 7),
                'min_samples_split': np.arange(1, 70, 7),
                'max_features': ['sqrt', 'log2']
            }
        },
        'Logistic Regression': {
            'model': LogisticRegression(),
            'param_dist': {
                'C': [0.1, 1.0, 10.0],
                'penalty': ['l1', 'l2']
            }
        },
        'KNN': {
            'model': KNeighborsClassifier(),
            'param_dist': {
                'n_neighbors': np.arange(3, 10),
                'weights': ['uniform', 'distance']
            }
        },
        'SVM': {
            'model': SVC(),
            'param_dist': {
                'C': [0.1, 1.0, 10.0],
                'kernel': ['linear', 'rbf', 'sigmoid']
            }
        }
    }
    
    for dataframe, dataframe_name in zip(all_dataframes, dataframes_names_str):
        X = dataframe[['my_rank', 'enemy_rank', 'rank_difference', 'is_higher',
                       'movesperplayer', 'Gametime', 'timepermove', 'white_kingcastle',
                       'white_queencastle', 'black_kingcastle', 'black_queencastle',
                       'White_moves_before_castle', 'Black_moves_before_castle', 'no opens',
                       'Black c4 e5 Nc3 Nf6', 'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6',
                       'Black d4 Nf6 c4 e6', 'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6',
                       'Black e4 e5 Bc4 Nc6', 'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6',
                       'Black e4 e5 Nf3 Nc6', 'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4',
                       'Black e4 e5 f4 d5', 'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5',
                       'Black e4 e6 Nc3 d5', 'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5',
                       'Black e4 e6 e5 c5', 'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6',
                       'White e4 c5 Nf3 d6', 'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5',
                       'White e4 c6 d4 d5', 'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6',
                       'White e4 d5 exd5 Qxd5', 'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6',
                       'White e4 e5 Nc3 d6', 'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6',
                       'White e4 e5 Nf3 Nf6', 'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5',
                       'White e4 e5 Nf3 d6', 'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5',
                       'White e4 g6 d4 Bg7']]
        y = dataframe['i_won']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
        
        for clf_name, clf_info in classifiers.items():
            clf = clf_info['model']
            param_dist = clf_info['param_dist']
            
            random_search = RandomizedSearchCV(clf, param_distributions=param_dist, cv=5, n_iter=10)
            random_search.fit(X_train, y_train)
            
            best_clf = random_search.best_estimator_
            print(f'Best Params for {clf_name} ({dataframe_name}):', random_search.best_params_)
            
            best_clf.fit(X_train, y_train)
            
            y_pred_train = best_clf.predict(X_train)
            y_pred = best_clf.predict(X_test)
            
            if clf_name not in final_results:
                final_results[clf_name] = {}
            
            if dataframe_name not in final_results[clf_name]:
                final_results[clf_name][dataframe_name] = {}
            
            final_results[clf_name][dataframe_name]["Trainging Set metrics"] = {
                'f1_score': f1_score(y_train, y_pred_train, average='weighted'),
                'precision_score': precision_score(y_train, y_pred_train, average='weighted'),
                'accuracy_score': accuracy_score(y_train, y_pred_train)
            }
            
            final_results[clf_name][dataframe_name]["Test Set metrics"] = {
                'f1_score': f1_score(y_test, y_pred, average='weighted'),
                'precision_score': precision_score(y_test, y_pred, average='weighted'),
                'accuracy_score': accuracy_score(y_test, y_pred)
            }
    
    return final_results

In [ ]:
final_res = model_for_different_gamemodes()